In [1]:
%%html
<style type='text/css'>
.CodeMirror{
    font-size: 16px;
    font-family: Monaco;
}

div.output_area pre {
    font-size: 12px;
}
</style>

In [2]:
import os
os.chdir(os.getcwd()+"./..")

In [3]:
from project_modules.utils import get_squad_v2_pandas_dataframe

squad_train_df, squad_dev_df = get_squad_v2_pandas_dataframe()

from sklearn.utils import shuffle

squad_train_df = shuffle(squad_train_df)

squad_train_df.head()

question = squad_train_df['question'].iloc[0]
context_list = squad_train_df.head(10)['context'].to_list()

,id,title,context,question,answer_text,answer_start,is_impossible
111315,5733e3164776f4190066142c,Premier_League,"However, by the turn of the 1990s the downward...",In which year was the Taylor Report published?,1990,280,False
65020,5726c22c5951b619008f7d6b,Cubism,The term Cubism did not come into general usag...,What year did the term Cubism become used more?,1911,54,False
5398,56d1e3efe7d4791d00902343,Buddhism,Suffering (Pāli: दुक्ख dukkha; Sanskrit दुःख d...,Suffering is a central concept in what?,Buddhism,82,False
90700,5729125daf94a219006aa028,Insect,"Humans regard certain insects as pests, and at...",Humans can control insects using what?,insecticides,74,False
5050,56d12aa917492d1400aabb24,Kanye_West,Music fans have turned to Change.org around th...,How many signatures were gathered for Kanye's ...,"50,000",435,False


In [4]:
from project_modules.config import document_selection_model_config

In [5]:
from project_modules.models import DocumentSelectionModelLoader,TwinAlBerts

In [6]:
model  = DocumentSelectionModelLoader(document_selection_model_config)

In [7]:
model.predict(question,context_list)

{'pred': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'prob': [0.95924914,
  0.103765585,
  0.076216474,
  0.037969753,
  0.039975353,
  0.04691883,
  0.037099537,
  0.08816648,
  0.050162114,
  0.03922937]}

In [8]:
from project_modules.utils import read_document_dict

In [9]:
from project_modules.config import flask_config, project_config

In [10]:
UPLOAD_FOLDER = project_config.upload_folder

In [11]:
document_dict = read_document_dict(UPLOAD_FOLDER)

In [12]:
title_document_list = [(title,"\n".join(sentence_list)) for title, sentence_list in document_dict.items()]

In [13]:
title_document_list

[('Apple Reports First Quarter Results',
  'CUPERTINO, CALIFORNIA JANUARY 27, 2022 Apple today announced financial results for its fiscal 2022 first quarter ended December 25, 2021. The Company posted an all-time revenue record of $123.9 billion, up 11 percent year over year, and quarterly earnings per diluted share of $2.10.\n“This quarter’s record results were made possible by our most innovative lineup of products and services ever,” said Tim Cook, Apple’s CEO. “We are gratified to see the response from customers around the world at a time when staying connected has never been more important. We are doing all we can to help build a better world — making progress toward our goal of becoming carbon neutral across our supply chain and products by 2030, and pushing forward with our work in education and racial equity and justice.”\n“The very strong customer response to our recent launch of new products and services drove double-digit growth in revenue and earnings, and helped set an all

In [14]:
context_list = [ item[1] for item in title_document_list]

In [15]:
question = "How much will the cash dividend be paid to  shareholders ?"

In [16]:
document_selection_dict = model.predict(question,context_list)

Token indices sequence length is longer than the specified maximum sequence length for this model (594 > 512). Running this sequence through the model will result in indexing errors


In [17]:
import numpy as np

In [18]:
doc_index = np.argmax(document_selection_dict['prob'])

In [19]:
pred_result = document_selection_dict['pred'][doc_index]

In [20]:
def predict_document(question, document_dict, document_selection_model):
    title_document_list = [(title, "\n".join(sentence_list))
                           for title, sentence_list in document_dict.items()]
    context_list = [item[1] for item in title_document_list]

    document_selection_dict = document_selection_model.predict(
        question, context_list)
    probs = document_selection_dict['prob']
    preds = document_selection_dict['pred']

    doc_index = np.argmax(probs)
    pred  = preds[doc_index]
    prob  = probs[doc_index]
    
    pred_document = None

    print("INFO: Maximum Document probablity:",prob)
    if pred==1:
        pred_document = title_document_list[doc_index][0]

        print("INFO: Predicted document:",pred_document)
    else:
        print("INFO: No any document to this questionn!")
    
    return pred_document

In [123]:
predict_document(question, document_dict,model)

INFO: Maximum Document probablity: 0.96036536
INFO: Predicted document: Apple Reports First Quarter Results


'Apple Reports First Quarter Results'

In [149]:
context_list = [item[1] for item in title_document_list]

In [150]:
sentence_list = list(document_dict.items())[0][1]

In [155]:
from textblob import TextBlob

In [156]:
context  = " ".join(sentence_list)

In [157]:
context

'CUPERTINO, CALIFORNIA JANUARY 27, 2022 Apple today announced financial results for its fiscal 2022 first quarter ended December 25, 2021. The Company posted an all-time revenue record of $123.9 billion, up 11 percent year over year, and quarterly earnings per diluted share of $2.10. “This quarter’s record results were made possible by our most innovative lineup of products and services ever,” said Tim Cook, Apple’s CEO. “We are gratified to see the response from customers around the world at a time when staying connected has never been more important. We are doing all we can to help build a better world — making progress toward our goal of becoming carbon neutral across our supply chain and products by 2030, and pushing forward with our work in education and racial equity and justice.” “The very strong customer response to our recent launch of new products and services drove double-digit growth in revenue and earnings, and helped set an all-time high for our installed base of active d

In [166]:
raw_sentence_list = ["sdfsdf",'asdfsaf.']

In [167]:
raw_sentence_list = [line+"." if not line.endswith(".") else line for line in raw_sentence_list]

In [168]:
raw_sentence_list 

['sdfsdf.', 'asdfsaf.']

In [164]:
def split_context_to_list(context, sentence_len=60):

    blob = TextBlob(context)a
    final_sentence_list = []
    accumulate_len = 0
    windows_sentences = []
    for sentence in blob.sentences:

        word_len = len(sentence.words)
        if accumulate_len+word_len <= sentence_len:
            if not sentence.endswith("."):
                windows_sentences.append(str(sentence)+". ")
            else:
                windows_sentences.append(str(sentence))
            accumulate_len += word_len
        else:
            windows_sentence = " ".join(windows_sentences)
            final_sentence_list.append(windows_sentence)
            windows_sentences = [str(sentence)]
            accumulate_len = word_len

    if len(windows_sentences) > 0:
        windows_sentence = " ".join(windows_sentences)
        final_sentence_list.append(windows_sentence)

    return final_sentence_list

In [165]:
split_context_to_list(context, sentence_len=60)

['CUPERTINO, CALIFORNIA JANUARY 27, 2022 Apple today announced financial results for its fiscal 2022 first quarter ended December 25, 2021. The Company posted an all-time revenue record of $123.9 billion, up 11 percent year over year, and quarterly earnings per diluted share of $2.10.',
 '“This quarter’s record results were made possible by our most innovative lineup of products and services ever,” said Tim Cook, Apple’s CEO. “We are gratified to see the response from customers around the world at a time when staying connected has never been more important.',
 'We are doing all we can to help build a better world — making progress toward our goal of becoming carbon neutral across our supply chain and products by 2030, and pushing forward with our work in education and racial equity and justice.” “The very strong customer response to our recent launch of new products and services drove double-digit growth in revenue and earnings, and helped set an all-time high for our installed base of

In [158]:
blob = TextBlob(context )

In [159]:
sentence_list = []

In [160]:
final_sentence_list = []

In [146]:
max_sentence_len = 60

In [147]:
accumulate_len = 0
windows_sentences =[]
for sentence in  blob.sentences:
    
    word_len = len(sentence.words)
    if accumulate_len+word_len <=max_sentence_len:
        windows_sentences.append(str(sentence))
        accumulate_len +=word_len
    else:
        windows_sentence = ". ".join(windows_sentences)
        final_sentence_list.append(windows_sentence)
        windows_sentences = [str(sentence)]
        accumulate_len = word_len

if len(windows_sentences)>0:
    windows_sentence = ". ".join(windows_sentences)
    final_sentence_list.append(windows_sentence)


In [148]:
final_sentence_list

['CUPERTINO, CALIFORNIA JANUARY 27, 2022 Apple today announced financial results for its fiscal 2022 first quarter ended December 25, 2021.',
 'The Company posted an all-time revenue record of $123.9 billion, up 11 percent year over year, and quarterly earnings per diluted share of $2.10..“This quarter’s record results were made possible by our most innovative lineup of products and services ever,” said Tim Cook, Apple’s CEO.',
 '“We are gratified to see the response from customers around the world at a time when staying connected has never been more important.',
 'We are doing all we can to help build a better world — making progress toward our goal of becoming carbon neutral across our supply chain and products by 2030, and pushing forward with our work in education and racial equity and justice.”.“The very strong customer response to our recent launch of new products and services drove double-digit growth in revenue and earnings, and helped set an all-time high for our installed bas

In [135]:
context

'GLG (Gerson Lehrman Group, Inc.) is a financial and global information services company headquartered in New York City. The company provides financial information and advises investors and consultants with business clients seeking expert advice.[2] It is the world\'s largest expert network, with over 900,000 freelance consultants.[3] GLG\'s experts include asset managers, investors, consultants, physicians, scientists, engineers, lawyers, senior current and former c-level executives, and former government members.[4][5] GLG\'s clients include strategy consulting corporations, hedge funds, private equity firms, professional service firms, and non-profit organizations.[6][7][8][9].The firm was founded in 1998 and has been backed by private equity firms Silver Lake Partners,[10] Bessemer Venture Partners,[11] and SFW Capital Partners.[12][9] GLG is headquartered in New York City, with offices in 22 cities in 12 countries. [9].History.Gerson Lehrman Group was founded by Yale Law School gr

In [ ]:
final_sentence_list

In [71]:
document_path  = r"static\uploaded_documents\Gerson_Lehrman_Group.txt"

In [72]:
with open(document_path, encoding='utf-8-sig') as f:
    document = f.readlines()
    raw_sentence_list = [line.strip()
                     for line in document if len(line.strip()) != 0]

In [75]:
context = "\n".join(raw_sentence_list)

In [77]:
def split_context_to_list(context,max_sentence_len = 60):
    
    blob = TextBlob(context )
    final_sentence_list = []
    accumulate_len = 0
    windows_sentences =[]
    for sentence in  blob.sentences:

        word_len = len(sentence.words)
        if accumulate_len+word_len <=max_sentence_len:
            windows_sentences.append(str(sentence))
            accumulate_len +=word_len
        else:
            windows_sentence = " ".join(windows_sentences)
            final_sentence_list.append(windows_sentence)
            windows_sentences = [str(sentence)]
            accumulate_len = word_len
            
    return final_sentence_list

In [82]:
split_context_to_list(context,max_sentence_len = 100)

["GLG (Gerson Lehrman Group, Inc.) is a financial and global information services company headquartered in New York City.. The company provides financial information and advises investors and consultants with business clients seeking expert advice.. [2] It is the world's largest expert network, with over 900,000 freelance consultants.. [3] GLG's experts include asset managers, investors, consultants, physicians, scientists, engineers, lawyers, senior current and former c-level executives, and former government members.. [4][5] GLG's clients include strategy consulting corporations, hedge funds, private equity firms, professional service firms, and non-profit organizations.",
 '[6][7][8][9]\nThe firm was founded in 1998 and has been backed by private equity firms Silver Lake Partners,[10] Bessemer Venture Partners,[11] and SFW Capital Partners.. [12][9] GLG is headquartered in New York City, with offices in 22 cities in 12 countries.. [9]\nHistory\nGerson Lehrman Group was founded by Ya

In [ ]:
clearn

In [ ]:
140

In [44]:
len(str(blob.sentences[0]).split(" "))

18

In [41]:
help(blob.sentences[0])

Help on Sentence in module textblob.blob object:

class Sentence(BaseBlob)
 |  Sentence(sentence, start_index=0, end_index=None, *args, **kwargs)
 |  
 |  A sentence within a TextBlob. Inherits from :class:`BaseBlob <BaseBlob>`.
 |  
 |  :param sentence: A string, the raw sentence.
 |  :param start_index: An int, the index where this sentence begins
 |                      in a TextBlob. If not given, defaults to 0.
 |  :param end_index: An int, the index where this sentence ends in
 |                      a TextBlob. If not given, defaults to the
 |                      length of the sentence - 1.
 |  
 |  Method resolution order:
 |      Sentence
 |      BaseBlob
 |      textblob.mixins.StringlikeMixin
 |      textblob.mixins.BlobComparableMixin
 |      textblob.mixins.ComparableMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, sentence, start_index=0, end_index=None, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signa

In [32]:
sentence_list

["GLG (Gerson Lehrman Group, Inc.) is a financial and global information services company headquartered in New York City. The company provides financial information and advises investors and consultants with business clients seeking expert advice.[2] It is the world's largest expert network, with over 900,000 freelance consultants.[3] GLG's experts include asset managers, investors, consultants, physicians, scientists, engineers, lawyers, senior current and former c-level executives, and former government members.[4][5] GLG's clients include strategy consulting corporations, hedge funds, private equity firms, professional service firms, and non-profit organizations.[6][7][8][9]",
 'The firm was founded in 1998 and has been backed by private equity firms Silver Lake Partners,[10] Bessemer Venture Partners,[11] and SFW Capital Partners.[12][9] GLG is headquartered in New York City, with offices in 22 cities in 12 countries. [9]',
 'History',
 'Gerson Lehrman Group was founded by Yale Law

In [23]:
context_list[1]

'GLG (Gerson Lehrman Group, Inc.) is a financial and global information services company headquartered in New York City. The company provides financial information and advises investors and consultants with business clients seeking expert advice.[2] It is the world\'s largest expert network, with over 900,000 freelance consultants.[3] GLG\'s experts include asset managers, investors, consultants, physicians, scientists, engineers, lawyers, senior current and former c-level executives, and former government members.[4][5] GLG\'s clients include strategy consulting corporations, hedge funds, private equity firms, professional service firms, and non-profit organizations.[6][7][8][9]\nThe firm was founded in 1998 and has been backed by private equity firms Silver Lake Partners,[10] Bessemer Venture Partners,[11] and SFW Capital Partners.[12][9] GLG is headquartered in New York City, with offices in 22 cities in 12 countries. [9]\nHistory\nGerson Lehrman Group was founded by Yale Law School